# Imports & Environment Setup

In [2]:
import os, random, math, numpy as np, pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from sklearn.metrics import classification_report, confusion_matrix

# Set Random Seeds

In [3]:
def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_all(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Loading Dataset - Dataset Paths (Train / Valid / Test)

In [ ]:
DATA_ROOT = "/kaggle/input/70-dog-breedsimage-data-set"

train_dir = os.path.join(DATA_ROOT, "train")
val_dir   = os.path.join(DATA_ROOT, "valid")
test_dir  = os.path.join(DATA_ROOT, "test")

print("Train exists:", os.path.isdir(train_dir))
print("Valid exists:", os.path.isdir(val_dir))
print("Test exists :", os.path.isdir(test_dir))


# Number of Classes

In [ ]:
def count_images(root):
    classes = sorted([d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))])
    counts = []
    for c in classes:
        cpath = os.path.join(root, c)
        n = len([f for f in os.listdir(cpath) if f.lower().endswith((".jpg",".jpeg",".png"))])
        counts.append((c, n))
    df = pd.DataFrame(counts, columns=["class", "count"]).sort_values("count", ascending=False)
    return df, classes

train_df, classes = count_images(train_dir)
val_df, _ = count_images(val_dir)
test_df, _ = count_images(test_dir)

print("Num classes:", len(classes))
print("Train images:", train_df["count"].sum())
print("Val images  :", val_df["count"].sum())
print("Test images :", test_df["count"].sum())

train_df.head()


# Class Distribution: Count Images per Breed

In [ ]:
plt.figure(figsize=(12,4))
plt.hist(train_df["count"], bins=20)
plt.title("Train: Distribution of images per class")
plt.xlabel("Images per class")
plt.ylabel("Number of classes")
plt.show()

train_df.describe()


# Visualize Random Training Samples

In [ ]:
def show_samples(root, classes, n=12):
    plt.figure(figsize=(12,8))
    for i in range(n):
        c = random.choice(classes)
        cdir = os.path.join(root, c)
        img_name = random.choice(os.listdir(cdir))
        img_path = os.path.join(cdir, img_name)
        img = Image.open(img_path).convert("RGB")
        plt.subplot(3,4,i+1)
        plt.imshow(img)
        plt.title(c, fontsize=9)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

show_samples(train_dir, classes, n=12)
